# Scraping artworks on wikiart by Selenium

### Author: Yangyu Wang
### Date: Jan 18, 2025

In [1]:
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

In [2]:
driver = webdriver.Firefox()

The geckodriver version (0.34.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (134.0.1); currently, geckodriver 0.35.0 is recommended for firefox 134.*, so it is advised to delete the driver in PATH and retry


In [3]:
def extract_artworkinfo(url):

    driver.get(url)

    artwork_info = {"url": url}
    
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.TAG_NAME, "h3"))
        )
    except:
        print("Error (404):", url)
        return url

    try:
        birth_date_element = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="dateCreated"]')
        artwork_info["Create Date"] = birth_date_element.text.strip()
    except:
        artwork_info["Create Date"] = None

    try:
        birth_place_element = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="locationCreated"]')
        artwork_info["Create Location"] = birth_place_element.text.strip()
    except:
        artwork_info["Create Location"] = None


    article = driver.find_element(By.XPATH, "//article")
    li_elements = article.find_elements(By.TAG_NAME, "li")

    for element in li_elements:
        try:
            field_name = element.find_element(By.TAG_NAME, "s").text.strip().replace(":", "")
        except:
            continue
            
        try:
            if field_name in ["Media", "Style", "Genre"]:
                value_elements = element.find_elements(By.XPATH, ".//a")
            else:
                value_elements = element.find_elements(By.XPATH, ".//span/a | .//a | .//span")
            value_texts = [v.text.strip() for v in value_elements if v.text.strip()]

            if value_texts:
                artwork_info[field_name] = "|".join(list(set(value_texts)))
        except:
            artwork_info[field_name] = None

        try:
            image_element = driver.find_element(By.XPATH, "//img[@itemprop='image']")
            image_url = image_element.get_attribute("src")
            artwork_info["image_url"] = image_url
            #artwork_info["image_n"] = n

            #image_data = requests.get(image_url).content
            #with open(f"artwork_picture/{n}.jpg", "wb") as file:
            #    file.write(image_data)
        except:
            artwork_info["image_url"] = None
            #artwork_info["image_n"] = None

    try:
        tags_elements = driver.find_elements(By.CSS_SELECTOR, ".tags-cheaps__item a.tags-cheaps__item__ref")
        tags = "|".join([tag.text.strip() for tag in tags_elements])
        artwork_info["tags"] = tags
    except:
        artwork_info["tags"] = None


    return artwork_info 


In [4]:
artworks = pd.read_csv("artist_data/artist_artwork.csv")

In [5]:
links = list(artworks["Link"])
links[0]

'https://www.wikiart.org/en/ancient-greek-pottery/attic-middle-geometric-amphora-from-kerameikos--800'

In [15]:
import os

url_found = []
for file in os.listdir("artwork_data"):
    if "revised_artwork_data" in file:
        artwork_data = pd.read_csv("artwork_data/" + file)
        url_found += list(artwork_data["url"])
        #artwork_data["Style"] = ["|".join(style.split("|")[0].split(", "))  if str(style) != "nan" else None for style in list(artwork_data["Style"])]
        #artwork_data["Genre"] = ["|".join(genre.split("|")[0].split(", "))  if str(genre) != "nan" else None for genre in list(artwork_data["Genre"])]
        #artwork_data["Media"] = ["|".join(media.split("|")[0].split(", "))  if str(media) != "nan" else None for media in list(artwork_data["Media"])]
        #artwork_data.to_csv("revised_" + file)

url_unfound = list(set(links) - set(url_found))

In [16]:
len(url_unfound)

104438

In [10]:
from tqdm import tqdm 
import time
import random

In [11]:
def extract_all_keys(dict_list):
    """
    Extract all unique keys from a list of dictionaries.
    Ensures that different dictionaries with different keys are fully accounted for.
    """
    ordered_keys = []
    seen_keys = set()
    
    for dictionary in dict_list:
        for key in dictionary.keys():
            if key not in seen_keys:
                ordered_keys.append(key)
                seen_keys.add(key)
                
    return ordered_keys

In [20]:
count = 7
count

7

In [14]:
artwork_data = []
notfound = []
n = 0

for url in tqdm(url_unfound, desc="Processing Artworks", unit="artwork"):
    try:
        artwork_info = extract_artworkinfo(url)
        if not isinstance(artwork_info, str):
            artwork_data.append(artwork_info)
        else:
            notfound.append((artwork_info))

    except KeyboardInterrupt:
        print("stopped")
        break
    except Exception:
        notfound.append((url))
    
    n += 1

    if n%10000 == 0:
        all_keys_artist = extract_all_keys(artwork_data)

        with open(f"artwork_data/revised_artwork_data_{count}.csv", mode='w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=all_keys_artist)

            writer.writeheader()

            for artist in tqdm(artwork_data, desc="Saving artist info", unit="artist"):
                writer.writerow({key: artist.get(key, "") for key in all_keys_artist})
        
        count += 1
        artwork_data = []

        if n//10000 == 1:
            break
        

Processing Artworks:   2%|▏         | 2084/134413 [1:00:30<58:05:41,  1.58s/artwork]

Error (404): https://www.wikiart.org/en/katsushika-hokusai/shimomeguro


Processing Artworks:   2%|▏         | 2912/134413 [1:24:47<53:28:05,  1.46s/artwork]

Error (404): https://www.wikiart.org/en/mikhail-larionov/summer-1912


Processing Artworks:   2%|▏         | 2985/134413 [1:26:50<53:48:21,  1.47s/artwork]

...

Error (404): https://www.wikiart.org/en/adolf-dietrich/sechs-meerschweinchen-im-stall-1934


Processing Artworks:  22%|██▏       | 30187/134413 [15:28:20<78:41:09,  2.72s/artwork]

Error (404): https://www.wikiart.org/en/pablo-picasso/bearded-man-1962


Processing Artworks:  23%|██▎       | 30432/134413 [15:36:12<53:18:51,  1.85s/artwork]

stopped
